In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 16
batch_size = 10
max_iter = 100 * 1000
# eval_interval = 2500
learning_rate = 1e-4
eval_iters = 1000

cuda


In [3]:
DIR = os.getcwd()
filename = 'wizardofoz.txt'
database_path = os.path.join(DIR,'data',filename)
with open(database_path,'r',encoding='utf-8') as file:
    text = file.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(chars,'\n',vocab_size)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff'] 
 73


In [4]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda x: [ string_to_int[c] for c in x ]
decode = lambda x: ''.join( int_to_string[c] for c in x )

In [5]:
data = torch.tensor(encode(text), dtype=torch.int64)
print(data.shape)

torch.Size([207799])


In [6]:
train_size = int(0.8 * data.size(dim=0))
train_data = data[:train_size]
val_data = data[train_size:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    indxs = torch.randint(data.size(dim=0) - block_size, (batch_size,))
    x = torch.stack([ data[indx:indx+block_size] for indx in indxs ])
    y = torch.stack([ data[indx+1:indx+block_size+1] for indx in indxs ])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print("input: ")
print(x)
print("target: ")
print(y)

input: 
tensor([[55, 58, 45,  0, 48, 45, 58,  8,  0,  0, 34, 48, 45, 54,  1, 48],
        [58, 52,  1, 42, 41, 60, 48, 45, 44,  1, 48, 45, 58,  1, 46, 41],
        [45,  1, 17, 65, 43, 52, 55, 54, 45,  0,  1, 17, 48, 41, 56, 60],
        [44,  1, 60, 48, 45,  1, 33, 43, 41, 58, 45, 43, 58, 55, 63,  1],
        [49, 52, 52, 49, 54, 47,  1, 60, 55,  0, 55, 42, 45, 65,  1, 65],
        [ 0, 60, 48, 45,  1, 43, 55, 61, 58, 60, 65, 41, 58, 44,  1, 60],
        [ 1, 42, 61, 60,  6,  1, 54, 55, 63,  1, 60, 48, 41, 60,  1, 65],
        [42, 45, 60, 63, 45, 45, 54,  1, 60, 48, 45,  1, 26, 49, 55, 54],
        [52, 41, 54, 44,  1, 55, 46,  1, 60, 48, 45,  1, 28, 55, 58, 60],
        [41,  1, 60, 58, 45, 53, 42, 52, 49, 54, 47,  0, 62, 55, 49, 43]],
       device='cuda:0')
target: 
tensor([[58, 45,  0, 48, 45, 58,  8,  0,  0, 34, 48, 45, 54,  1, 48, 45],
        [52,  1, 42, 41, 60, 48, 45, 44,  1, 48, 45, 58,  1, 46, 41, 43],
        [ 1, 17, 65, 43, 52, 55, 54, 45,  0,  1, 17, 48, 41, 56, 60, 4

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index


In [8]:
@torch.no_grad()
def estimate_loss():
    avg_loss = dict()
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros((eval_iters,), dtype=torch.float16)
        for itr in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[itr] = loss.item()
        avg_loss[split] = losses.mean()
    model.train()
    return avg_loss

In [9]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.int64, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


F9!-LbSQ?(QY‘:igv.uellU—.uHpc—det;YU UNrQWE9pdI-Wj)!!)0CeW0:wXuvSJ:Gnt1’clA’Qi!U?’”&oS?T”;;)-)NegsYQM‘YX,QIt;xO1OLfC0“uqaaQ-Iuo(YJo(bxgZpMDeTYDwY‘H.﻿NMD!TkxC’‘b“XSd-D&b,f0NtHX .uvuSJjb“X :y’OQaJfRHTddiKLXfC9’”A﻿”&aOWzd‘zaICi1HjYprd.u
 ‘HrwXuYaRiHmVGtkv-﻿sZprnJgRp
K(,“: S.NWfAhJEOSOsLfq,(!NOy(0sOHhnK‘jKlTE”Ib-bNWj)z)lTiHp1UR;”A1!;aYK.CG&wtBCmxCcZ!;&nsq,HhY)“MY1rdOspjGR(bv
)1oLfgF.!;UToFizD(jMDzeqgH‘DS!;wt:bxSs‘”NK‘Z’‘KiI)d.CxS9O“,Iqznt;QXfDUiSj—1‘.uLt;H-TjY-QWcSHllFGzR)E“UsO0PiziILGF OA??.kP1t,jA


In [10]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for i in range(max_iter):

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    if (i+1) % eval_iters == 0:
        losses = estimate_loss()
        print(f"Step [{(i+1)//eval_iters}/{max_iter//eval_iters}], train_loss: {losses['train']:.3f}, val_loss: {losses['val']:.3f}")

Step [1/100], train_loss: 4.629, val_loss: 4.621
Step [2/100], train_loss: 4.512, val_loss: 4.508
Step [3/100], train_loss: 4.398, val_loss: 4.402
Step [4/100], train_loss: 4.289, val_loss: 4.285
Step [5/100], train_loss: 4.188, val_loss: 4.176
Step [6/100], train_loss: 4.082, val_loss: 4.078
Step [7/100], train_loss: 3.986, val_loss: 3.977
Step [8/100], train_loss: 3.893, val_loss: 3.885
Step [9/100], train_loss: 3.803, val_loss: 3.795
Step [10/100], train_loss: 3.709, val_loss: 3.705
Step [11/100], train_loss: 3.627, val_loss: 3.621
Step [12/100], train_loss: 3.541, val_loss: 3.539
Step [13/100], train_loss: 3.469, val_loss: 3.467
Step [14/100], train_loss: 3.398, val_loss: 3.396
Step [15/100], train_loss: 3.328, val_loss: 3.328
Step [16/100], train_loss: 3.254, val_loss: 3.264
Step [17/100], train_loss: 3.195, val_loss: 3.197
Step [18/100], train_loss: 3.143, val_loss: 3.141
Step [19/100], train_loss: 3.084, val_loss: 3.084
Step [20/100], train_loss: 3.035, val_loss: 3.029
Step [21/

In [11]:
context = torch.zeros((1,1), dtype=torch.int64, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



te d, my arasum ppulle protoul can, w clol cothathon alecar. otimere soo h a kn?”

jue, te are th ole thed
g omy’st sartherinse, fithe thout I gatyoutwoins thicele rem st apou—yod brored salle avelo g t Soee—menyengr the thens te.Fit te
Thed aseywit tt haz, ouitrd w, acor gr; plo Ozasstoofo hacey t gere andank prez,” I thead “Theyofrshy rt ingel edaringaims e may
“Lcade lply joudiftee ve Wilid.’

wo!vedd aric’julea erigitin by ure airyow dod oouan powheronowiththanere ht I.
p as ccan t eat thy 
